In [7]:
%pip install import-ipynb

Note: you may need to restart the kernel to use updated packages.


In [8]:
from scipy.spatial import distance_matrix
import re
import import_ipynb
import numpy as np
import solver

In [9]:
def load_instance(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Dictionary of values
    values = {}
    
    # Lists to save coordinates and weights
    coordinates = []
    weights = []

    for line in lines:
        if ',' in line:
            key, value = line.strip().split(',')
            if key in ["NUM DRONES", "NUM TRUCKS"]:
                values[key] = int(value)
            elif key in ["TRUCK CAP", "DRONE CAP", "TRUCK SPEED", "DRONE SPEED", "DRONE ENDURANCE", "DRONE TIME LIMIT", "TRUCK TIME LIMIT", "TRUCK UNIT COST", "DRONE UNIT COST"]:
                values[key] = float(value)
        else:
            parts = re.sub(' +',' ', line).split()[1:] 
            # Add coordinates and weights to the lists
            x, y, weight = map(float, parts)
            coordinates.append((x, y))
            weights.append(weight)

    return values, coordinates, weights

In [10]:
def distance_matrix_computation(coord):
    coord_array = np.array(coord)
    return distance_matrix(coord_array, coord_array)

In [11]:
def test_solver(file_path):
    values, coordinates, weights = load_instance(file_path)
    d_matrix= distance_matrix_computation(coordinates)
    t_t_matrix = d_matrix/values["TRUCK SPEED"]
    t_d_vector = d_matrix[0]/values["DRONE SPEED"]
    model = solver.PDSVRPModel(C_T= values["TRUCK UNIT COST"], 
                    C_D=values["DRONE UNIT COST"],
                    D=values["NUM DRONES"],
                    h=values["NUM TRUCKS"],
                    distances= d_matrix,
                    t_t= t_t_matrix,
                    t_d= t_d_vector,
                    Q_t= values["TRUCK CAP"],
                    Q_d= values["DRONE CAP"],
                    T_t= values["TRUCK TIME LIMIT"],
                    T_d= values["DRONE TIME LIMIT"],
                    w=weights)
    model.build_model()
    model.solve()
    model.print_results()
    

In [12]:
test_solver("instances/50-r-2-e.txt")

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2706 rows, 5350 columns and 23004 nonzeros
Model fingerprint: 0x295037be
Variable types: 2600 continuous, 2750 integer (2750 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+03]
  Objective range  [4e-02, 3e+01]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 1e+03]
Presolve removed 50 rows and 1323 columns
Presolve time: 0.04s
Presolved: 2656 rows, 4027 columns, 23361 nonzeros
Variable types: 1325 continuous, 2702 integer (2702 binary)

Root relaxation: objective 5.590958e+01, 446 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0